In [ ]:
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

print("Torch version: ", torch. __version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)


####################################################################
# Prepare Data
####################################################################

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

train_set = torchvision.datasets.MNIST('.data/', train=True, download=True, transform=transform)
#? Considera usar transform=transforms.Compose([ToTensor(), Normalize((0.1307,), (0.3081,))]) para centrar/escala antes del flatten.
#train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST('.data/', train=False, download=True, transform=transform)
#test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

print("Train images: ", train_set)
print("Image: ", train_set[0][0])
print("Label: ", train_set[0][1])
print("Label one hot: ", F.one_hot(torch.tensor(train_set[0][1]), num_classes=10))


####################################################################
# Dataset Class
####################################################################

class MNIST_dataset(Dataset):

    def __init__(self, data, partition = "train"):

        print("\nLoading MNIST ", partition, " Dataset...")
        self.data = data
        self.partition = partition
        print("\tTotal Len.: ", len(self.data), "\n", 50*"-")

    def __len__(self):
        return len(self.data)

    def from_pil_to_tensor(self, image):
        return torchvision.transforms.ToTensor()(image)

    def __getitem__(self, idx):

        # Image
        image_tensor = self.data[idx][0].view(-1)
        # PIL Image to torch tensor
        #! image_tensor = self.from_pil_to_tensor(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        #! image_tensor = image_tensor.view(-1)
#? Tambien puedes normalizar aqui (image_tensor = (image_tensor - mean) / std) si no usas transforms.

        # Label
        label = torch.tensor(self.data[idx][1])
        # label = F.one_hot(label, num_classes=10).float()
        label = torch.tensor(self.data[idx][1], dtype=torch.long)
#? Alternativa: devuelve label como entero y usa CrossEntropyLoss(label_smoothing=0.1) para regularizar sin one-hot.

        return {"img": image_tensor, "label": label}

train_dataset = MNIST_dataset(train_set, partition="train")
test_dataset = MNIST_dataset(test_set, partition="test")


####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = 0
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)
#? Para GPU ayuda pin_memory=True y persistent_workers=True cuando num_workers>0.

####################################################################
# Early stopping Class
####################################################################

import copy
import torch

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0, mode="max"):
        """
        patience: nº de epochs sin mejora para parar
        min_delta: mejora mínima para considerar 'mejora real'
        mode: "max" si monitorizas accuracy, "min" si monitorizas loss
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None
        self.counter = 0
        self.best_state_dict = None
        self.best_epoch = -1

    def step(self, score, model, epoch):
        if self.best_score is None:
            self.best_score = score
            self.best_state_dict = copy.deepcopy(model.state_dict())
            self.best_epoch = epoch
            return False  # no parar

        improved = (score > self.best_score + self.min_delta) if self.mode == "max" \
                   else (score < self.best_score - self.min_delta)

        if improved:
            self.best_score = score
            self.best_state_dict = copy.deepcopy(model.state_dict())
            self.best_epoch = epoch
            self.counter = 0
        else:
            self.counter += 1

        return self.counter >= self.patience  # True => parar


####################################################################
# Neural Network Class
####################################################################

# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        #* AÑADIDO CAPA BATCHNORM1D Y DROPOUT
        self.linear1 = nn.Linear(784, 512)
        self.relu1 = nn.ReLU()
        self.BatchNorm1d1 = nn.BatchNorm1d(512)
        self.drop1 = nn.Dropout(0.2)
        self.linear2 = nn.Linear(512, 256)
        self.BatchNorm1d2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.3)
        self.linear3 = nn.Linear(256, 128)
        self.BatchNorm1d3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.4)
        self.classifier = nn.Linear(128, num_classes)
#? BatchNorm1d tras cada Linear y Dropout(0.1-0.3) antes de la activacion suelen mejorar la generalizacion.
#? Un MLP mas profundo pero mas estrecho (ej. 784->512->256->128->10) reduce parametros y overfitting sin usar CNN.

    def forward(self, x):
        out = self.drop1(self.relu1(self.BatchNorm1d1(self.linear1(x))))
        out = self.drop2(self.relu2(self.BatchNorm1d2(self.linear2(out))))
        out = self.drop3(self.relu3(self.BatchNorm1d3(self.linear3(out))))
        out = self.classifier(out)
        return out


# Instantiating the network and printing its architecture
num_classes = 10
net = Net(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, weight_decay=1e-6, momentum=0.9) # Original lr=0.01
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-2)
epochs = 75 # Original = 25
#? Prueba AdamW con weight_decay mas alto (p.ej. 1e-2) y un scheduler CosineAnnealingLR u OneCycleLR.


####################################################################
# Training
####################################################################

# Load model in GPU
net.to(device)

print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0

early_stopper = EarlyStopping(patience=10, min_delta=0.1, mode="max")
# min_delta=0.05 significa +0.05% de accuracy como mejora mínima (ajústalo si quieres)

for epoch in range(epochs):


    # TRAIN NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:

            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()
#? Puedes recortar gradientes con torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0) si ves inestabilidad.

            # Update gradients
            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

    train_loss /= len(train_dataloader.dataset)

    # TEST NETWORK
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
      with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
          for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)

    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
        epoch + 1, train_loss, test_loss, 100. * train_correct / len(train_dataloader.dataset), test_accuracy
    ))

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch

        # Save best weights
        torch.save(net.state_dict(), "best_model.pt")
        
    should_stop = early_stopper.step(test_accuracy, net, epoch)
    print(f"EarlyStopping: best={early_stopper.best_score:.2f}% (epoch {early_stopper.best_epoch+1}) "
        f"patience_counter={early_stopper.counter}/{early_stopper.patience}")

    if should_stop:
        print(f"Stopping early at epoch {epoch+1}. Best was epoch {early_stopper.best_epoch+1} "
            f"with acc {early_stopper.best_score:.2f}%")
        break

#? Agrega early stopping con paciencia (p.ej. 10 epocas) y ReduceLROnPlateau para bajar lr cuando el val loss se estanque.

print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)

# So far:
# best acc:  98.24 (default)
# best acc:  96.64 with lr: 0.001
# best acc:  98.26 with 2 hidden layers
# best acc:  98.64 with lr: 0.1
# best acc:  98.02 with lr: 0.001 & 75 epochs

####################################################################
# Load best weights
####################################################################

# Load best weights
net.load_state_dict(torch.load("best_model.pt"))

test_loss, test_correct = 0, 0
net.eval()
with torch.no_grad():
    with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)
print("Final best acc: ", test_accuracy)

Torch version:  2.10.0+cu130
Device:  cuda
Train images:  Dataset MNIST
    Number of datapoints: 60000
    Root location: .data/
    Split: Train
Image:  <PIL.Image.Image image mode=L size=28x28 at 0x2A4841C8E30>
Label:  5
Label one hot:  tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers 0
Net(
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (relu1): ReLU()
  (BatchNorm1d1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (BatchNorm1d2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (linear3): Linear(in_features=256, out_

Test 0: 100%|██████████| 100/100 [00:00<00:00, 112.48batch/s]


[Epoch 1] Train Loss: 0.002914 - Test Loss: 0.001043 - Train Accuracy: 92.09% - Test Accuracy: 96.69%
EarlyStopping: best=96.69% (epoch 1) patience_counter=0/10


Test 1: 100%|██████████| 100/100 [00:00<00:00, 112.74batch/s]


[Epoch 2] Train Loss: 0.001324 - Test Loss: 0.000782 - Train Accuracy: 96.09% - Test Accuracy: 97.40%
EarlyStopping: best=97.40% (epoch 2) patience_counter=0/10


...

[Epoch 17] Train Loss: 0.000309 - Test Loss: 0.000571 - Train Accuracy: 99.06% - Test Accuracy: 98.37%
EarlyStopping: best=98.36% (epoch 8) patience_counter=9/10


Test 17: 100%|██████████| 100/100 [00:00<00:00, 104.33batch/s]


[Epoch 18] Train Loss: 0.000287 - Test Loss: 0.000620 - Train Accuracy: 99.03% - Test Accuracy: 98.36%
EarlyStopping: best=98.36% (epoch 8) patience_counter=10/10
Stopping early at epoch 18. Best was epoch 8 with acc 98.36%

BEST TEST ACCURACY:  98.39  in epoch  13


Test 17: 100%|██████████| 100/100 [00:00<00:00, 107.47batch/s]

Final best acc:  98.39
